<a href="https://colab.research.google.com/github/sarbagyaneu123/Color_Detection/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Tokenize with improved config
vocab_size = 10000
oov_tok = "<OOV>"

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data['message'])

X_train = tokenizer.texts_to_sequences(train_data['message'])
X_test = tokenizer.texts_to_sequences(test_data['message'])

maxlen = 120  # Slightly increased
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)

y_train = train_data['label'].values
y_test = test_data['label'].values


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=6, validation_data=(X_test, y_test), verbose=2)


In [ ]:
def predict_message(pred_text):
  seq = tokenizer.texts_to_sequences([pred_text])
  padded = tf.keras.preprocessing.sequence.pad_sequences(seq, padding='post', maxlen=maxlen)
  prediction_prob = model.predict(padded, verbose=0)[0][0]
  label = 'spam' if prediction_prob > 0.5 else 'ham'
  return [float(prediction_prob), label]


In [ ]:
# Step 6: Test the Function
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(f"❌ Failed on: {msg}")
      print(f"   Predicted: {prediction}")
      passed = False

  if passed:
    print("✅ You passed the challenge. Great job!")
  else:
    print("❗ You haven't passed yet. Keep trying.")

test_predictions()
